In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from data.download_wrds_data import main as get_wrds
import pickle
import os

%matplotlib inline
%load_ext autoreload
%autoreload 2

In [3]:
# get_wrds(id_type="tic")

Enter your WRDS username [joaoromeuferraz]:jrferraz14
Enter your password:········
WRDS recommends setting up a .pgpass file.
Create .pgpass file now [y/n]?: y
Created .pgpass file successfully.
Loading library list...
Done


In [2]:
from data.pull_data import pull_wrds_sample_data
from mom_trans.data_prep import deep_momentum_strategy_features, include_changepoint_features

In [3]:
prices = pickle.load(open("data/prices.p", "rb"))
constituents = pickle.load(open("data/constituents.p", "rb"))

prices["datadate"] = pd.to_datetime(prices["datadate"])

In [5]:
cpd_module_folder = "../../../Desktop/OneDrive/cool stuff/data/"

In [16]:
from settings.default import CPD_WRDS_OUTPUT_FOLDER, ONEDRIVE_CPD_WRDS_OUTPUT_FOLDER

In [12]:
tickers = [x[:-4] for x in os.listdir(os.path.join(cpd_module_folder, "yfinance_cpd_21lbw"))]

lookback_window_length=21
extra_lbw = [126]
output_name="base"
id_type="tic"

In [13]:
features = pd.concat(
        [
            deep_momentum_strategy_features(pull_wrds_sample_data(ticker, prices, id_type)).assign(
                ticker=ticker
            )
            for ticker in tickers
        ]
    )

/Users/joaoromeuferraz/Documents/GitHub/momentum_transformer/mom_trans/data_prep.py:120: FutureWarning: weekofyear and week have been deprecated, please use DatetimeIndex.isocalendar().week instead, which returns a Series.  To exactly reproduce the behavior of week and weekofyear and return an Index, you may call pd.Int64Index(idx.isocalendar().week)
  df_asset["week_of_year"] = df_asset.index.weekofyear
/Users/joaoromeuferraz/Documents/GitHub/momentum_transformer/mom_trans/data_prep.py:120: FutureWarning: weekofyear and week have been deprecated, please use DatetimeIndex.isocalendar().week instead, which returns a Series.  To exactly reproduce the behavior of week and weekofyear and return an Index, you may call pd.Int64Index(idx.isocalendar().week)
  df_asset["week_of_year"] = df_asset.index.weekofyear
/Users/joaoromeuferraz/Documents/GitHub/momentum_transformer/mom_trans/data_prep.py:120: FutureWarning: weekofyear and week have been deprecated, please use DatetimeIndex.isocalendar()

In [15]:
features.date = features.index
features.index.name = "Date"

In [21]:
cpd_dir = lambda lbw: os.path.join(cpd_module_folder, f"yfinance_cpd_{lbw}lbw")

In [27]:
lookback_window_length = [21, 126]

In [28]:
features_backup = features.copy()

In [32]:
x1 = include_changepoint_features(features_backup, cpd_dir(21), 21)

In [39]:
features = features_backup.copy()

In [40]:
for lbw in lookback_window_length:
    features = include_changepoint_features(features, cpd_dir(lbw), lbw)

In [41]:
features

,close,srs,daily_returns,daily_vol,target_returns,norm_daily_return,norm_monthly_return,norm_quarterly_return,norm_biannual_return,norm_annual_return,...,day_of_month,week_of_year,month_of_year,year,date,ticker,cp_rl_21,cp_score_21,cp_rl_126,cp_score_126
Date,,,,,,,,,,,,,,,,,,,,,
1996-01-31,4.625000,4.625000,0.012158,0.032494,0.008733,0.374162,0.604516,0.287668,1.544587,2.773520,...,31,5,1,1996,1996-01-31,CSCO,0.525251,0.845105,0.380880,0.868050
1996-02-01,4.763889,4.763889,0.030030,0.032298,-0.004265,0.929790,1.139506,0.267348,1.768489,2.760811,...,1,5,2,1996,1996-02-01,CSCO,0.617405,0.947214,0.444298,0.830404
1996-02-02,4.694444,4.694444,-0.014577,0.031954,0.011373,-0.456191,1.341640,0.157711,1.554574,2.704598,...,2,5,2,1996,1996-02-02,CSCO,0.619980,0.899936,0.396520,0.851787
1996-02-05,4.875000,4.875000,0.038462,0.032023,0.005885,1.201060,1.548730,0.163443,1.606667,2.745955,...,5,6,2,1996,1996-02-05,CSCO,0.375134,0.926491,0.404718,0.857386
1996-02-06,4.972222,4.972222,0.019943,0.031603,-0.002713,0.631050,1.822173,0.276863,1.716709,2.844674,...,6,6,2,1996,1996-02-06,CSCO,0.514018,0.965610,0.412632,0.839750
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2002-02-22,19.750000,19.750000,0.023316,0.028304,-0.002197,0.823777,-0.094469,-0.032442,0.899586,0.178812,...,22,8,2,2002,2002-02-22,INTU,0.712723,0.732645,0.949639,1.000000
2002-02-25,19.620000,19.620000,-0.006582,0.027868,-0.004061,-0.236195,-0.104353,-0.161103,0.972094,0.287487,...,25,9,2,2002,2002-02-25,INTU,0.750626,0.999689,1.000000,1.000000
2002-02-26,19.385000,19.385000,-0.011978,0.027500,-0.012230,-0.435542,-0.138754,-0.239649,0.785991,0.426247,...,26,9,2,2002,2002-02-26,INTU,0.476217,0.826014,0.966121,1.000000


In [24]:
features_w_cpd = include_changepoint_features(features, cpd_dir(lookback_window_length), lookback_window_length)

In [25]:
extra = extra_lbw[0]

In [19]:
extra_data = pd.read_csv()

'data/features'

In [21]:
cpd_module_folder.replace(
                        f"wrds_cpd_{lookback_window_length}lbw.csv",
                        f"wrds_cpd_{extra}lbw.csv",
                    )

'../../../Desktop/OneDrive/cool stuff/data/yfinance_cpd_21lbw'